# First Assignment
## Segmenting Neighborhods in Toronto

# Part 1

In [1]:
import numpy as np
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### Scrapping wikipedia table and preparing dataset

In [2]:
df = pd.read_html(url, header = 0)[0]
df = df[df.Borough != 'Not assigned']
df_new = df
df_new['Neighbourhood'] = df.groupby('Postcode')['Neighbourhood'].transform(lambda x : ','.join(x))
df_new = df_new[['Postcode','Borough','Neighbourhood']].drop_duplicates()
df_new = df_new.sort_index()
print(df_new.shape)
df_new.head()



(103, 3)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Not assigned


 # Part 2

### Now lets add coordinates to corresponding postal codes

Geocode did not work as expected so I used the csv file provided

In [3]:
#Reading dataframe from
df_coord = pd.read_csv('http://cocl.us/Geospatial_data')
#Changing name of Postal card column to allow for merge between dataframes
df_coord = df_coord.rename(columns={'Postal Code': 'Postcode'})
#Merge two dataframes
df_ready = pd.merge(df_new, df_coord, on='Postcode')
#Display dataframe
df_ready = df_ready.sort_values('Postcode').reset_index(drop = True)
df_ready.head()
    

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


 # Part 3
 ## Remaining steps:
 ### Plotting data
 ### Segmenting
 ### Completing witth Foursquare API
 ### Clustering

### Import remaining necessay libraries

In [4]:
#Importing remaining necessary libraries
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from geopy.geocoders import Nominatim
import requests


### Prepare coordinates for Toronto map creation

In [5]:

address = 'Toronto, Ontario'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

/home/renato/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


### Use information to plot Toronto Map

In [6]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_ready['Latitude'], df_ready['Longitude'], df_ready['Borough'], df_ready['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

 ## Lets segment and cluster only Neighbourhoods that contain the Toronto word
 ### This step is for simplification purposes

In [7]:
toronto_neighbourhoods = df_ready[df_ready['Neighbourhood'].str.contains('Toronto')].reset_index(drop=True)
toronto_neighbourhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763
1,M4J,East York,East Toronto,43.685347,-79.338106
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752
4,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576


Lets plot the result

In [8]:
map_toronto_neighbourhoods = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighbourhoods['Latitude'], toronto_neighbourhoods['Longitude'], toronto_neighbourhoods['Borough'], toronto_neighbourhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_neighbourhoods)  
    
map_toronto_neighbourhoods

### Using Foursquare API

In [9]:
CLIENT_ID = 'TE1CMG0YKTGUDCAHUV5WYWYRWNES5B3FVLMYRC2BND5EEIPU' # your Foursquare ID
CLIENT_SECRET = 'UF4E1AZNHSKO05CU2RMZWOJIHSBYXX2Y1LN2IEAVRXGMS0X3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100



### Defining function to get nearby venues

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
toronto_venues = getNearbyVenues(names=toronto_neighbourhoods['Neighbourhood'],
                                   latitudes=toronto_neighbourhoods['Latitude'],
                                   longitudes=toronto_neighbourhoods['Longitude']
                                  )
toronto_venues.head()

CFB Toronto,Downsview East
East Toronto
North Toronto West
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Harbord,University of Toronto
Humber Bay Shores,Mimico South,New Toronto


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto,Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto,Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
2,East Toronto,43.685347,-79.338106,Aldwych Park,43.684901,-79.341091,Park
3,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
4,East Toronto,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store


Now let's group the venues by neighbourhood

In [12]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CFB Toronto,Downsview East",2,2,2,2,2,2
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,3,3,3,3,3,3
"Harbord,University of Toronto",35,35,35,35,35,35
"Harbourfront East,Toronto Islands,Union Station",100,100,100,100,100,100
"Humber Bay Shores,Mimico South,New Toronto",14,14,14,14,14,14
North Toronto West,22,22,22,22,22,22


## Neighborhood analyzis

Applying one-hot encoding to venue categories

In [13]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix = "", prefix_sep = "")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
#move neighborhood column to first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])

toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,Baseball Stadium,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bakery,Bank,Bar,...,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,"CFB Toronto,Downsview East",0.000000,0.5,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
1,"Design Exchange,Toronto Dominion Centre",0.000000,0.0,0.020000,0.00,0.01,0.02,0.020000,0.00,0.030000,...,0.03,0.00,0.000000,0.02,0.01,0.010000,0.01,0.01,0.000000,0.01
2,East Toronto,0.000000,0.0,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
3,"Harbord,University of Toronto",0.000000,0.0,0.000000,0.00,0.00,0.00,0.057143,0.00,0.057143,...,0.00,0.00,0.028571,0.00,0.00,0.028571,0.00,0.00,0.028571,0.00
4,"Harbourfront East,Toronto Islands,Union Station",0.000000,0.0,0.000000,0.05,0.01,0.00,0.020000,0.01,0.020000,...,0.01,0.01,0.010000,0.01,0.00,0.010000,0.01,0.01,0.000000,0.01
5,"Humber Bay Shores,Mimico South,New Toronto",0.000000,0.0,0.071429,0.00,0.00,0.00,0.071429,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00
6,North Toronto West,0.045455,0.0,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00


In [15]:
toronto_grouped.shape

(7, 100)

#### Function to sort venues by descending order

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Using defined function to create dataframe with top 10 venues for each neighborhood in Toronto

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"CFB Toronto,Downsview East",Airport,Park,Wine Bar,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
1,"Design Exchange,Toronto Dominion Centre",Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Steakhouse,Gastropub,Seafood Restaurant,Deli / Bodega,Bar
2,East Toronto,Park,Convenience Store,Wine Bar,Event Space,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
3,"Harbord,University of Toronto",Café,Japanese Restaurant,Italian Restaurant,Bakery,Restaurant,Bar,Bookstore,Noodle House,Comfort Food Restaurant,College Gym
4,"Harbourfront East,Toronto Islands,Union Station",Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Italian Restaurant,Restaurant,Pizza Place


# Cluster Neighborhoods

Lets cluster the neighborhoods into 5 clusters with k-means

In [18]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [19]:


toronto_merged = toronto_neighbourhoods
toronto_merged = toronto_merged.rename(columns = {"Neighbourhood": "Neighborhood"})
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged


,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,1,Airport,Park,Wine Bar,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
1,M4J,East York,East Toronto,43.685347,-79.338106,2,Park,Convenience Store,Wine Bar,Event Space,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,3,Clothing Store,Coffee Shop,Sporting Goods Shop,Salon / Barbershop,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop,Park
3,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Italian Restaurant,Restaurant,Pizza Place
4,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Steakhouse,Gastropub,Seafood Restaurant,Deli / Bodega,Bar
5,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,0,Café,Japanese Restaurant,Italian Restaurant,Bakery,Restaurant,Bar,Bookstore,Noodle House,Comfort Food Restaurant,College Gym
6,M8V,Etobicoke,"Humber Bay Shores,Mimico South,New Toronto",43.605647,-79.501321,4,Café,Seafood Restaurant,Pharmacy,American Restaurant,Pizza Place,Fried Chicken Joint,Bakery,Gym,Sandwich Place,Business Service


### Now lets plot the neighborhoods with corresponding cluster color

In [20]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining Clusters

Cluster 1

In [21]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Downtown Toronto,0,Coffee Shop,Hotel,Aquarium,Café,Brewery,Fried Chicken Joint,Scenic Lookout,Italian Restaurant,Restaurant,Pizza Place
4,Downtown Toronto,0,Coffee Shop,Café,Hotel,Italian Restaurant,Restaurant,Steakhouse,Gastropub,Seafood Restaurant,Deli / Bodega,Bar
5,Downtown Toronto,0,Café,Japanese Restaurant,Italian Restaurant,Bakery,Restaurant,Bar,Bookstore,Noodle House,Comfort Food Restaurant,College Gym


Cluster 2

In [22]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Airport,Park,Wine Bar,Fast Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop


Cluster 3

In [23]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East York,2,Park,Convenience Store,Wine Bar,Event Space,Comfort Food Restaurant,Concert Hall,Cosmetics Shop,Dance Studio,Deli / Bodega,Dessert Shop


Cluster 4

In [24]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,3,Clothing Store,Coffee Shop,Sporting Goods Shop,Salon / Barbershop,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop,Park


Cluster 5

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Etobicoke,4,Café,Seafood Restaurant,Pharmacy,American Restaurant,Pizza Place,Fried Chicken Joint,Bakery,Gym,Sandwich Place,Business Service
